"""
Tiedosto: 
Tekijä Petri Lamminaho  
Kuvaus: Koodi joka tekee yksinkertaisen päätöksentekopuun
        (Decision Tree) alusta asti Ilman ulkoisia kirjastoja  
         Käyttää CART algoritmia
"""

In [1]:
#Testi data 
data = [
    ['vihrea', 3, 'silea', 'omena'],
    ['keltainen', 3, 'silea', 'omena'],
    ['punainen', 1,'silea', 'rypale'],
    ['keltainen', 3,'karhea', "sitruuna"],
    ['punainen', 1, 'karhea', 'mansikka']
]

In [2]:
data

[['vihrea', 3, 'silea', 'omena'],
 ['keltainen', 3, 'silea', 'omena'],
 ['punainen', 1, 'silea', 'rypale'],
 ['keltainen', 3, 'karhea', 'sitruuna'],
 ['punainen', 1, 'karhea', 'mansikka']]

In [3]:
sarake_headers = ['vari', 'ymparysmitta cm', 'pinta', 'label']

In [4]:
sarake_headers

['vari', 'ymparysmitta cm', 'pinta', 'label']

In [5]:
def palauta_luokkien_lkm(rivit): #palauttaa jokaisen luokassa olevan "luokan" nimen ja lkm 
    counts = {}  
    for rivi in rivit:
        label = rivi[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


In [6]:
palauta_luokkien_lkm(data)

{'mansikka': 1, 'omena': 2, 'rypale': 1, 'sitruuna': 1}

In [7]:
def onko_numero(value):          # jos muuttuja on numero palauttaa True Muutoin False  
    return isinstance(value, int) or isinstance(value, float)

In [8]:
#test pitäisi true 
onko_numero(10)

True

In [9]:
#tulostaa falsen 
onko_numero("keltainen")

False

In [10]:
#Kysymys-luokkka  
class Kysymys:
    def __init__(self, col, val):
        self.col = col
        self.val = val

    def vertaa_onko_match(self, example): 
        val = example[self.col]

        if onko_numero(val):
            return val >= self.val
        else:
            return val == self.val

    def __repr__(self):
        condition = "=="
        if onko_numero(self.val):
            condition = ">="
        return "Onko %s %s %s?" % (
            sarake_headers[self.col], condition, str(self.val))


In [11]:
Kysymys(0, "keltainen")

Onko vari == keltainen?

In [12]:
Kysymys(1,3)

Onko ymparysmitta cm >= 3?

In [13]:
def jaa_puu_true_false(rivit, kysymys): # jakaa puun nodet True tai False listoihin kysymyksien perusteella 
    true_taulukko = []
    false_taulukko = []
    for rivi in rivit:
        if kysymys.vertaa_onko_match(rivi):
            true_taulukko.append(rivi)
        else:
            false_taulukko.append(rivi)
    return true_taulukko, false_taulukko

In [14]:
true_taulukko, false_taulukko = jaa_puu_true_false(data, Kysymys(1, 3))

In [15]:
true_taulukko

[['vihrea', 3, 'silea', 'omena'],
 ['keltainen', 3, 'silea', 'omena'],
 ['keltainen', 3, 'karhea', 'sitruuna']]

In [16]:
false_taulukko

[['punainen', 1, 'silea', 'rypale'], ['punainen', 1, 'karhea', 'mansikka']]

In [17]:
def gini(rivit):#Palauttaa tiedon miten järjestyksessä puu on 
    lkm = palauta_luokkien_lkm(rivit)
    print(lkm)
    impurity = 1 #"epäpuhtaus"
    for lbl in lkm:
         prob_of_lbl = lkm[lbl] / float(len(rivit))
         impurity -= prob_of_lbl**2
    return impurity

In [18]:
kaikki_samaa = [['omena'],
              ['omena'],
               ['omena']]

gini(kaikki_samaa)
#impurity on 0 

{'omena': 3}


0.0

In [19]:
pieni_sekoitus = [['omena'],
               ['rypale']]
# impurity 0.5 
gini(pieni_sekoitus)

{'omena': 1, 'rypale': 1}


0.5

In [20]:
suuri_sekoitus = [['omena'],
                  ['sitruuna'],
                  ['rypale'],
                  ['mansikka'],
                  ['banaani'],
                  #['paaryma']
                    ]
gini(suuri_sekoitus)

{'omena': 1, 'sitruuna': 1, 'rypale': 1, 'mansikka': 1, 'banaani': 1}


0.7999999999999998

In [21]:
def gain_info(vasen, oikea,epavarmuus):# testaa mikä on paras kysymys kulloisellekin nodelle  
    p = float(len(vasen))/(len(vasen)+ len(oikea)) 
    return epavarmuus - p * gini(vasen)-(1 - p) * gini(oikea)

In [22]:
epavarmuus = gini(data)

{'omena': 2, 'rypale': 1, 'sitruuna': 1, 'mansikka': 1}


In [23]:
epavarmuus

0.7199999999999999

In [24]:
true_rivit, false_rivit = jaa_puu_true_false(data, Kysymys(0, 'vihrea'))

In [25]:
gain_info(true_rivit, false_rivit, epavarmuus)

{'omena': 1}
{'omena': 1, 'rypale': 1, 'sitruuna': 1, 'mansikka': 1}


0.11999999999999977

In [26]:
true_rivit

[['vihrea', 3, 'silea', 'omena']]

In [27]:
true_rivit, false_rivit = jaa_puu_true_false(data, Kysymys(0, 'punainen'))

In [28]:
gain_info(true_rivit, false_rivit, epavarmuus)

{'rypale': 1, 'mansikka': 1}
{'omena': 2, 'sitruuna': 1}


0.25333333333333313

In [29]:
true_rivit

[['punainen', 1, 'silea', 'rypale'], ['punainen', 1, 'karhea', 'mansikka']]

In [30]:
def etsi_paras_jako(rivit): # Selvittää miten puu saadaan jaettua parhaiten. Palauttaa kysymyksen ja arvon kuinka paljon sillä saadaan infoa 
    paras_gain = 0 
    paras_kysymys = None
    tamanhetkinen_epavarmuus = gini(rivit)
    num_features = len(rivit[0])-1
    
    for col in range(num_features):
        values = set([rivi[col] for rivi in rivit])
        for val in values:
            kysymys = Kysymys(col, val)
            
            true_rivit, false_rivit = jaa_puu_true_false(rivit,kysymys)
            if len(true_rivit) == 0 or len(false_rivit) == 0 :
                continue
                
            gain = gain_info (true_rivit, false_rivit, tamanhetkinen_epavarmuus)
            
            if gain >=  paras_gain:
                paras_gain, paras_kysymys = gain, kysymys
                
    return paras_gain, paras_kysymys          

In [31]:
best_gain, best_question = etsi_paras_jako(data)
best_gain

{'omena': 2, 'rypale': 1, 'sitruuna': 1, 'mansikka': 1}
{'omena': 1, 'sitruuna': 1}
{'omena': 1, 'rypale': 1, 'mansikka': 1}
{'omena': 1}
{'omena': 1, 'rypale': 1, 'sitruuna': 1, 'mansikka': 1}
{'rypale': 1, 'mansikka': 1}
{'omena': 2, 'sitruuna': 1}
{'omena': 2, 'sitruuna': 1}
{'rypale': 1, 'mansikka': 1}
{'omena': 2, 'rypale': 1}
{'sitruuna': 1, 'mansikka': 1}
{'sitruuna': 1, 'mansikka': 1}
{'omena': 2, 'rypale': 1}


0.2533333333333332

In [32]:
best_question

Onko pinta == silea?

In [33]:
class Lehti: # Lehti node Node josta ei päästä enää alaspäin.  Vastaus 
    
    def __init__(self, rivit):
        self.ennustukset = palauta_luokkien_lkm(rivit)    

In [34]:
class Paatos_Node: #tavallinen solmu. Pitää tiedon kysymyksestä mikä kysytään ja child nodeistaan(vasen ja  oikea)
    def __init__(self,
                 kysymys,
                 true_oksa,
                 false_oksa):
        self.kysymys = kysymys
        self.true_oksa = true_oksa
        self.false_oksa = false_oksa
    

In [35]:
def tee_puu(data):# luo puun rekiusrsiviisesti 
    gain, kysymys = etsi_paras_jako(data)
    if gain == 0:
        return Lehti(data)
   
    true_rivit, false_rivit = jaa_puu_true_false(data, kysymys)
    true_oksa = tee_puu(true_rivit)
    false_oksa = tee_puu(false_rivit)
    return Paatos_Node(kysymys, true_oksa, false_oksa)

In [36]:
def tulosta_puu(node, valimerkki=" "): # tulostaa puun
    if isinstance(node, Lehti):
        print(valimerkki + "Ennuste", node.ennustukset)
        return
    
    # tulostetaan kysymykset 
    print(valimerkki + str(node.kysymys))
    
    #tulostetaan true_oksat
    print(valimerkki + '--> True:')
    tulosta_puu(node.true_oksa, valimerkki + " ")
    
    #tulostetaan false_oksat
    print(valimerkki + '--> False:')
    tulosta_puu(node.false_oksa, valimerkki + " ")
    
    

In [37]:
mun_puu = tee_puu(data)

{'omena': 2, 'rypale': 1, 'sitruuna': 1, 'mansikka': 1}
{'omena': 1, 'sitruuna': 1}
{'omena': 1, 'rypale': 1, 'mansikka': 1}
{'omena': 1}
{'omena': 1, 'rypale': 1, 'sitruuna': 1, 'mansikka': 1}
{'rypale': 1, 'mansikka': 1}
{'omena': 2, 'sitruuna': 1}
{'omena': 2, 'sitruuna': 1}
{'rypale': 1, 'mansikka': 1}
{'omena': 2, 'rypale': 1}
{'sitruuna': 1, 'mansikka': 1}
{'sitruuna': 1, 'mansikka': 1}
{'omena': 2, 'rypale': 1}
{'omena': 2, 'rypale': 1}
{'omena': 1}
{'omena': 1, 'rypale': 1}
{'omena': 1}
{'omena': 1, 'rypale': 1}
{'rypale': 1}
{'omena': 2}
{'omena': 2}
{'rypale': 1}
{'omena': 2}
{'omena': 1}
{'omena': 1}
{'omena': 1}
{'omena': 1}
{'rypale': 1}
{'sitruuna': 1, 'mansikka': 1}
{'sitruuna': 1}
{'mansikka': 1}
{'mansikka': 1}
{'sitruuna': 1}
{'sitruuna': 1}
{'mansikka': 1}
{'sitruuna': 1}
{'mansikka': 1}


In [38]:
tulosta_puu(mun_puu)

 Onko pinta == silea?
 --> True:
  Onko ymparysmitta cm >= 3?
  --> True:
   Ennuste {'omena': 2}
  --> False:
   Ennuste {'rypale': 1}
 --> False:
  Onko ymparysmitta cm >= 3?
  --> True:
   Ennuste {'sitruuna': 1}
  --> False:
   Ennuste {'mansikka': 1}


In [39]:
def clasify( rivi, node): # luokittelija funktio Eli antaa vastauksen 
    if isinstance(node, Lehti):
        return node.ennustukset
    if node.kysymys.vertaa_onko_match(rivi):
        return clasify(rivi, node.true_oksa)
    else:
        return clasify(rivi, node.false_oksa)

In [40]:
print(clasify(data[4], mun_puu))

{'mansikka': 1}


In [41]:
def tulosta_ennuste_luotettavuus(lkm):# tulostaa vastauksen ja vastauksen oikeellisuuden todennäköisuuden prosenteina 
    total = sum(lkm.values()) * 1.0
    probs = {}
    for lbl in lkm.keys():
        probs[lbl] = str(int(lkm[lbl] / total * 100)) + "%"
    return probs

In [42]:
tulosta_ennuste_luotettavuus(clasify(data[0], mun_puu))

{'omena': '100%'}

In [43]:
tulosta_ennuste_luotettavuus(clasify(data[1], mun_puu))

{'omena': '100%'}

In [44]:
tulosta_ennuste_luotettavuus(clasify(data[4], mun_puu))

{'mansikka': '100%'}